In [0]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 28263 (delta 0), reused 1 (delta 0), pack-reused 28260
Receiving objects: 100% (28263/28263), 25.61 MiB | 25.68 MiB/s, done.
Resolving deltas: 100% (19642/19642), done.


In [0]:
%cd transformers
!pip install .

/content/transformers
Processing /content/transformers
     |████████████████████████████████| 3.8MB 2.8MB/s 
     |████████████████████████████████| 1.1MB 29.7MB/s 
     |████████████████████████████████| 890kB 30.5MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=666257 sha256=686e44fe4a95e859194ee113f95a5f5e42b9d0b2be382b46d9e005eefbaea346
  Stored in directory: /tmp/pip-ephem-wheel-cache-mc6p3o_r/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b8c0159f011fc6114b3aa91fe36b74c3b1ba98c20f56229c3cc4a4195d528832
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import pandas as pd

In [0]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/NLP_Project_Folder/data/Microsoft_Challenge

/content/gdrive/My Drive/Colab Notebooks/NLP_Project_Folder/data/Microsoft_Challenge


In [0]:
!ls

mc500.dev.ans  mc500.test.ans  mc500.train.ans
mc500.dev.tsv  mc500.test.tsv  mc500.train.tsv


Paper : https://www.aclweb.org/anthology/D13-1020.pdf



Dataset : https://github.com/mcobzarenco/mctest/tree/master/data/MCTest

In [0]:
df_train_answers = pd.read_csv('mc500.train.ans', sep="\t", header=None)

In [0]:
df_train_answers.head()

,0,1,2,3
0,B,C,C,D
1,B,D,D,B
2,D,D,D,B
3,A,B,C,C
4,C,C,D,A


In [0]:
df_train = pd.read_csv('mc500.train.tsv', sep="\t", header=None)

In [0]:
df_train.shape

(300, 23)

In [0]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,mc500.train.0,Author: 2778399758;Work Time(s): 839;Qual. sco...,Alyssa got to the beach after a long trip. She...,one: What city is Alyssa in?,trip,Miami,Atlanta,beach,one: Why did Alyssa go to Miami?,swim,travel,visit friends,laying out,multiple: How many friends does Alyssa have?,1,2,3,4,multiple: What did Alyssa eat at the restaurant?,steak,soup,salad,catfish
1,mc500.train.1,Author: 3208685851;Work Time(s): 836;Qual. sco...,"One morning, Elena woke up, much like she did ...",multiple: What is the very first thing Elena d...,she says hello to the tree,she throws the covers on the floor,she says hello to the sun,she gets out of bed,one: At what time of day does this story take ...,Before the tree,At the end of the day,The story doesn't say,Morning,multiple: What happened to Mr. Fish in the end?,"He got put back in the bowl, but he was dead",He started swimming around in Elena's hand,He got put in a new bowl,He got put back in his bowl and started swimmi...,multiple: What was missing from the fish bowl?,the rocks,Mr. Fish,the water,the tiny castle
2,mc500.train.2,Author: 3165469218;Work Time(s): 1546;Qual. sc...,Billy was like a king on the school yard. A ki...,multiple: Billy was like a...,prince,dummy,queen,king,multiple: Who was Billy?,The skinny kid,A teacher,A little kid,The big kid,one: What did the boy grab from his home?,a rock,a bench,a tree,Fishing line,one: What did the boy call Billy?,A dork,Dummy,Mr.Stupid Kid,A bully
3,mc500.train.3,Author: 3323630398;Work Time(s): 712;Qual. sco...,Billy went to the farm to buy some beef for hi...,one: What did Billy buy at the farm?,Beef,Chicken,Cows,Fence,multiple: What color were the spots on the cows?,Blue,Brown,White,Black,multiple: How many chickens were on the purple...,Ten,Six,Five,Four,one: How did Billy's brother feel after dinner?,Strange,Scared,Happy,Sad
4,mc500.train.4,Author: 646501657;Work Time(s): 743;Qual. scor...,The road to Grandpa's house was long and windi...,multiple: Why did Grandpa answer the door?,Because he saw the insects,Because Jimmy was walking,Because Jimmy knocked,Because the trip took a long time,one: Where do Jimmy and his Grandpa sit?,On insects,Outside,By the fire,On the path,multiple: What was Jimmy's favorite insect in ...,Beetle,Lady bug,Moth,Lightning bug,one: Why doesn't Jimmy collect more moths?,They're difficult to catch and they take up a ...,They're his favorite,He doesn't like them,Grandpa doesn't like them


In [0]:
df_train.columns = ['id', 'Author', 'Text', 'Q1', 'Q1A', 'Q1B', 'Q1C', 'Q1D', 'Q2', 'Q2A', 'Q2B', 'Q2C', 'Q2D', 'Q3', 'Q3A', 'Q3B', 'Q3C', 'Q3D', 'Q4', 'Q4A', 'Q4B', 'Q4C', 'Q4D']

In [0]:
import difflib
def similarity_calculator(x, y):
    seq = difflib.SequenceMatcher(None, x, y)  # Gestalt Pattern Matching
    d = seq.ratio()*100
    return d 

In [0]:
from transformers import  pipeline, QuestionAnsweringPipeline, AutoTokenizer, AutoModelForQuestionAnswering, AutoModel, AutoModelWithLMHead

In [0]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
# ktrapeznikov/albert-xlarge-v2-squad-v2


{
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

Bert Model Paper: https://arxiv.org/pdf/1810.04805.pdf

In [0]:
nlp_qa = pipeline('question-answering',model=model, tokenizer=tokenizer)

In [0]:
nlp_qa(context='Hugging Face is a French company based in New-York.', question='Where is based Hugging Face ?')

{'answer': 'New-York.', 'end': 50, 'score': 0.985049841952879, 'start': 42}

In [0]:
typical = ['A', 'B', 'C', 'D']

In [0]:
total = 0
correct = 0

In [0]:
for (index1, data_text), (index2, data_answer) in zip(df_train.iterrows(), df_train_answers.iterrows()):
    id_ = data_text['id'].split('.')[-1]
    text = data_text['Text']
    # summary = summarizer(text)[0]['summary_text']
    for i in range(1,5):
        q = data_text['Q{}'.format(i)].split(': ')[-1]
        q_ans = data_text['Q{i}{correct}'.format(i=i, correct=data_answer[i-1])]
        possible_answers = [data_text['Q{num}{s}'.format(num=i, s=s )] for s in typical]
        model_answer = nlp_qa(context=text, question=q)['answer']
        score_list=[]
        for p in possible_answers:
            score_list.append(similarity_calculator(model_answer, p))
        last_answer = possible_answers[score_list.index(max(score_list))]
        if last_answer == q_ans:
            correct += 1
        total += 1

In [0]:
print((correct/total)*100)

70.16666666666667


In [0]:
"""Alyssa got to the beach after a long trip. She's from Charlotte. She traveled from Atlanta. She's now in Miami. She went to Miami to visit some friends. But she wanted some time to herself at the beach, so she went there first. After going swimming and laying out, she went to her friend Ellen's house. Ellen greeted Alyssa and they both had some lemonade to drink. Alyssa called her friends Kristin and Rachel to meet at Ellen's house. The girls traded stories and caught up on their lives. It was a happy time for everyone. The girls went to a restaurant for dinner. The restaurant had a special on catfish. Alyssa enjoyed the restaurant's special. Ellen ordered a salad. Kristin had soup. Rachel had a steak. After eating, the ladies went back to Ellen's house to have fun. They had lots of fun. They stayed the night because they were tired. Alyssa was happy to spend time with her friends again.
Question: What city is Alyssa in?
Correct Answer: Miami
['trip', 'Miami', 'Atlanta', 'beach']
Model Answer: Miami.
Question: Why did Alyssa go to Miami?
Correct Answer: visit friends
['swim', 'travel', 'visit friends', 'laying out']
Model Answer: to visit some friends.
Question: How many friends does Alyssa have?
Correct Answer: 3
['1', '2', '3', '4']
Model Answer: Kristin and Rachel
Question: What did Alyssa eat at the restaurant?
Correct Answer: catfish
['steak', 'soup', 'salad', 'catfish']
Model Answer: catfish."""

In [0]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/NLP_Project_Folder/data

/content/gdrive/My Drive/Colab Notebooks/NLP_Project_Folder/data


In [0]:
!ls

Microsoft_Challenge  Natural_Questions	netflix.txt  Swag_Data	twitter.txt


In [0]:
with open('netflix.txt') as f:
    context = f.read().replace('\n','') 

In [0]:
context

'Netflix provides a personalized subscription service that allows our members to access movies and TV shows (“Netflix content”) streamed over the Internet to certain Internet-connected TVs, computers and other devices ("Netflix ready devices").These Terms of Use govern your use of our service. As used in these Terms of Use, "Netflix service", "our service" or "the service" means the personalized service provided by Netflix for discovering and watching Netflix content, including all features and functionalities, recommendations and reviews, the website, and user interfaces, as well as all content and software associated with our service.Membership1.1. Your Netflix membership will continue until terminated. To use the Netflix service you must have Internet access and a Netflix ready device, and provide us with one or more Payment Methods. “Payment Method” means a current, valid, accepted method of payment, as may be updated from time to time, and which may include payment through your ac

In [0]:
%%time
nlp_qa(context=context, question='When can I cancel Netflix?')['answer']

CPU times: user 5min 4s, sys: 719 ms, total: 5min 5s
Wall time: 5min 6s


'at any time,'

In [0]:
%%time
nlp_qa(context=context, question='When should I cancel Netflix?')['answer']

CPU times: user 4min 40s, sys: 677 ms, total: 4min 41s
Wall time: 4min 42s


'before your billing date,'

In [0]:
%%time
nlp_qa(context=context, question='How old should I be to use Netflix?')['answer']

CPU times: user 4min 45s, sys: 686 ms, total: 4min 46s
Wall time: 4min 47s


'18'

In [0]:
%%time 
nlp_qa(context=context, question='What happens if I am not older than 18 ?')['answer']

CPU times: user 5min 4s, sys: 696 ms, total: 5min 5s
Wall time: 5min 6s


'Minors may only use the service under the supervision of an adult.4.2.'

In [0]:
def read_terms():
    context = input('Please provide Terms:  ')
    question = input('What would you like to know ?:')
    print(nlp_qa(context=context, question=question))

In [0]:
read_terms()

Please provide Terms:  3.4. Cancellation. You can cancel your Netflix membership at any time, and you will continue to have access to the Netflix service through the end of your billing period. To the extent permitted by the applicable law, payments are non-refundable and we do not provide refunds or credits for any partial membership periods or unwatched Netflix content. To cancel, go to the "Account" page and follow the instructions for cancellation. If you cancel your membership, your account will automatically close at the end of your current billing period. To see when your account will close, click "Billing details" on the "Account" page. If you signed up for Netflix using your account with a third party as a Payment Method and wish to cancel your Netflix membership, you may need to do so through such third party, for example by visiting your account with the applicable third party and turning off auto-renew, or unsubscribing from the Netflix service through that third party. You

In [0]:
read_terms()

Please provide Terms:  3.4. Cancellation. You can cancel your Netflix membership at any time, and you will continue to have access to the Netflix service through the end of your billing period. To the extent permitted by the applicable law, payments are non-refundable and we do not provide refunds or credits for any partial membership periods or unwatched Netflix content. To cancel, go to the "Account" page and follow the instructions for cancellation. If you cancel your membership, your account will automatically close at the end of your current billing period. To see when your account will close, click "Billing details" on the "Account" page. If you signed up for Netflix using your account with a third party as a Payment Method and wish to cancel your Netflix membership, you may need to do so through such third party, for example by visiting your account with the applicable third party and turning off auto-renew, or unsubscribing from the Netflix service through that third party. You

In [0]:
with open('twitter.txt') as f:
    context_twitter = f.read().replace('\n','') 

In [0]:
%%time
nlp_qa(context=context_twitter, question='How old should I be to use Twitter?')['answer']

CPU times: user 1min 3s, sys: 597 ms, total: 1min 3s
Wall time: 1min 4s


'13 years old,'

In [0]:
%%time
nlp_qa(context=context_twitter, question='Can I use Twitter if I am not older than 13?')['answer']

CPU times: user 1min 36s, sys: 585 ms, total: 1min 37s
Wall time: 1min 38s


'you must be at least 13 years old,'